In [ ]:
# This reloads python code imported from *.py files automatically
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import tqdm 
import tqdm.notebook
from optimization import *
import time


In [ ]:
n = 100
n_iter = 1000
distr = [G1, G2]
lambdas = [0.1, 0.25, 0.7, 0.95]

n_cities = 10
n_seeds = 2

mutation_strategies = [0, 1, 3]
init_prob = 0.0
betas = [1, 2, 5, 50]

# Evaluate over init_prob, beta

results = {}

total_runs = len(distr) * len(lambdas) * n_cities * n_seeds * len(mutation_strategies) * len(betas)
progress_bar = tqdm.notebook.tqdm(range(total_runs))
    
for d in distr:
    results[d.__name__] = {}
    ld = results[d.__name__]
    for lambd in lambdas:
        md = {}
        ld[lambd] = md
        for mutation in mutation_strategies:
            md[mutation] = {}
            
        for beta in betas:
            for mutation in mutation_strategies:
                md[mutation][beta] = []
            
            # Generate a current cities and different seeds
            for s in range(n_cities):
                np.random.seed(s)
                cities = d(n)
                for seed in range(n_seeds):
                    for mutation in mutation_strategies:
                        idx = n_seeds * s + seed
                        np.random.seed(idx)
                        sel_c, sel_conv, loss, loss_convex = optimize(cities,
                                                                      lambd, 
                                                                      beta=beta, 
                                                                      n_iter=n_iter,
                                                                      mutation_strategy=mutation, 
                                                                      initial_selection_probability=init_prob, 
                                                                      verbose=False)
                        md[mutation][beta].append(loss_convex)
                        progress_bar.update()


In [ ]:
# Plot results: Plot for each lambda the different mutation strategies
# for different values of beta

fig, ax = plt.subplots(len(distr), len(lambdas), figsize=(12, 8))

for i, d in enumerate(distr):

    for j, l in enumerate(lambdas):

        for beta_idx, beta in enumerate(betas):

            means = np.zeros(len(mutation_strategies))
            std = np.zeros(len(mutation_strategies))
            for k, m in enumerate(mutation_strategies):
                means[k] = np.mean(results[d.__name__][l][m][beta])
                std[k] = np.std(results[d.__name__][l][m][beta])

            curr_ax = ax[i] if len(lambdas) == 1 else ax[i, j]
            curr_ax.errorbar(np.arange(len(mutation_strategies)) + beta_idx /
                             len(betas) * 0.1, means, std[k], linestyle='None', marker='^')
#             plt.errorbar(x, y, e, linestyle='None', marker='^')
            curr_ax.set_title(f'{d.__name__}, $\lambda = {l}$')
            curr_ax.legend(betas, title='beta')
            curr_ax.set_xticks(np.arange(len(mutation_strategies)))
            curr_ax.set_xlabel("Mutation strategy")
            curr_ax.set_ylabel("Average loss (lower is better)")

fig.tight_layout(pad=2)